# AG2: GroupChat (Resuming a Group Chat)

Resuming a Group Chat
- https://docs.ag2.ai/docs/user-guide/advanced-concepts/groupchat/resuming-group-chat

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Resuming a Group Chat

```python
# Save chat messages for resuming later on using the chat history
messages_json = mygroupchatmanager.messages_to_string(previous_chat_result.chat_history)

# Alternatively you can use the GroupChat's messages property
messages_json = mygroupchatmanager.messages_to_string(mygroupchatmanager.groupchat.messages)

# example of the JSON string
[{"content": "Find the latest paper about gpt-4 on arxiv and find its potential applications in software.", "role": "user", "name": "Admin"}, {"content": "Plan:\n1. **Engineer**: Search for the latest paper on GPT-4 on arXiv.\n2. **Scientist**: Read the paper and summarize the key findings and potential applications of GPT-4.\n3. **Engineer**: Identify potential software applications where GPT-4 can be utilized based on the scientist's summary.\n4. **Scientist**: Provide insights on the feasibility and impact of implementing GPT-4 in the identified software applications.\n5. **Engineer**: Develop a prototype or proof of concept to demonstrate how GPT-4 can be integrated into the selected software application.\n6. **Scientist**: Evaluate the prototype, provide feedback, and suggest any improvements or modifications.\n7. **Engineer**: Make necessary revisions based on the scientist's feedback and finalize the integration of GPT-4 into the software application.\n8. **Admin**: Review the final software application with GPT-4 integration and approve for further development or implementation.\n\nFeedback from admin and critic is needed for further refinement of the plan.", "role": "user", "name": "Planner"}, {"content": "Agree", "role": "user", "name": "Admin"}, {"content": "Great! Let's proceed with the plan outlined earlier. I will start by searching for the latest paper on GPT-4 on arXiv. Once I find the paper, the scientist will summarize the key findings and potential applications of GPT-4. We will then proceed with the rest of the steps as outlined. I will keep you updated on our progress.", "role": "user", "name": "Planner"}]
```

## Example of resuming a GroupChat

In [2]:
import os
import autogen

# Put your api key in the environment variable OPENAI_API_KEY
config_list = [
    {
        "api_type": "openai",
        # "model": "gpt-4o",
        "model": "gpt-4o-mini",
        "api_key": os.environ["OPENAI_API_KEY"],
    }
]

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

In [3]:
# Create Agents, GroupChat, and GroupChatManager in line with the original group chat

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=gpt4_config,
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor],
    messages=[],
    max_round=10,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [4]:
# Messages in a JSON string
previous_state = r"""[{"content": "Find the latest paper about gpt-4 on arxiv and find its potential applications in software.", "role": "user", "name": "Admin"}, {"content": "Plan:\n1. **Engineer**: Search for the latest paper on GPT-4 on arXiv.\n2. **Scientist**: Read the paper and summarize the key findings and potential applications of GPT-4.\n3. **Engineer**: Identify potential software applications where GPT-4 can be utilized based on the scientist's summary.\n4. **Scientist**: Provide insights on the feasibility and impact of implementing GPT-4 in the identified software applications.\n5. **Engineer**: Develop a prototype or proof of concept to demonstrate how GPT-4 can be integrated into the selected software application.\n6. **Scientist**: Evaluate the prototype, provide feedback, and suggest any improvements or modifications.\n7. **Engineer**: Make necessary revisions based on the scientist's feedback and finalize the integration of GPT-4 into the software application.\n8. **Admin**: Review the final software application with GPT-4 integration and approve for further development or implementation.\n\nFeedback from admin and critic is needed for further refinement of the plan.", "role": "user", "name": "Planner"}, {"content": "Agree", "role": "user", "name": "Admin"}, {"content": "Great! Let's proceed with the plan outlined earlier. I will start by searching for the latest paper on GPT-4 on arXiv. Once I find the paper, the scientist will summarize the key findings and potential applications of GPT-4. We will then proceed with the rest of the steps as outlined. I will keep you updated on our progress.", "role": "user", "name": "Planner"}]"""

In [5]:
# Prepare the group chat for resuming
last_agent, last_message = manager.resume(messages=previous_state)

# Resume the chat using the last agent and message
# result = last_agent.initiate_chat(recipient=manager, message=last_message, clear_history=False)
chat_result = last_agent.initiate_chat(recipient=manager, message=last_message, clear_history=False)

# Output the final chat history showing the original 4 messages and resumed messages
for i, message in enumerate(groupchat.messages):
    print(
        f"#{i + 1}, {message['name']}: {message['content'][:80]}".replace("\n", " "),
        f"{'...' if len(message['content']) > 80 else ''}".replace("\n", " "),
    )

Prepared group chat with 4 messages, the last speaker is Planner
Planner (to chat_manager):

Great! Let's proceed with the plan outlined earlier. I will start by searching for the latest paper on GPT-4 on arXiv. Once I find the paper, the scientist will summarize the key findings and potential applications of GPT-4. We will then proceed with the rest of the steps as outlined. I will keep you updated on our progress.

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

As an engineer, I will now search for the latest paper on GPT-4 on arXiv. I will write a Python script to automate this search process. 

Here is the code to find the latest paper on GPT-4 from arXiv:

```python
import requests

def fetch_latest_gpt4_paper():
    url = "http://export.arxiv.org/api/query"
    query = "gpt-4"
    params = {
        'search_query': query,
        'start': 0,
        'max_results': 1,
        'sortBy': 'lastUpd

InternalServerError: Error code: 500 - {'error': {'message': 'The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.', 'type': 'model_error', 'param': None, 'code': None}}

In [ ]:
print(chat_result.summary)

In [ ]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

## Example of resuming a terminated GroupChat

In [10]:
# Previous state with TERMINATE in the last message
previous_state = r"""[{"content": "Let's get this meeting started. We'll have a set order of speakers. First the Product_Manager will create 3 new product ideas. Then the Chief_Marketing_Officer will speak and talk about the idea they have chosen to move forward with. Then the Digital_Marketer will create 3 marketing strategies for that idea. We MUST stick to the speaking order which is Product_Manager first, Chief_Marketing_Officer second, Digital_Marketer third, and finally the Chief_Marketing_Officer will speak and end the meeting.", "role": "user", "name": "Chairperson"}, {"content": "Sounds like a plan! Let's get started.  As the Product_Manager, I'd like to present three new product ideas for our luxury car models:  1. 'EcoLux': A hybrid electric-gasoline vehicle that combines the best of both worlds, offering exceptional fuel efficiency and reduced carbon footprint. 2. 'AeroSport': A high-performance sports car with advanced aerodynamics, designed to deliver unparalleled speed and agility on the track or on the open road. 3. 'SmartDrive': An intelligent driver-assistance system that uses AI-powered sensors and cameras to anticipate and respond to potential hazards, ensuring a safer driving experience.  Now it's your turn, Chief_Marketing_Officer! Which of these ideas do you think has the most potential?", "role": "user", "name": "Chief_Marketing_Officer"}, {"content": "Thank you for presenting those innovative product ideas, Product_Manager.  After careful consideration, I believe 'EcoLux' has the most potential. With the growing concern about climate change and environmental sustainability, a hybrid electric-gasoline vehicle that offers exceptional fuel efficiency and reduced carbon footprint could be a game-changer in the luxury car market. Additionally, it aligns with our company's commitment to innovation and responsibility.  Now it's your turn, Digital_Marketer! Can you come up with three marketing strategies for 'EcoLux'?", "role": "user", "name": "Product_Manager"}, {"content": "Thank you, Chief_Marketing_Officer!  For 'EcoLux', I propose the following three marketing strategies:  1. 'Green Revolution' Campaign: Highlighting the eco-friendly features of EcoLux through a series of social media ads and influencer partnerships. We can partner with eco-conscious influencers to showcase how EcoLux is not only a luxury car but also an environmentally responsible choice. 2. 'Fuel for Thought' Content Series: Creating a content series that explores the intersection of technology, sustainability, and luxury. This could include blog posts, videos, and podcasts that delve into the innovative features of EcoLux and its impact on the environment. 3. 'EcoLux Experience' Event Marketing: Hosting exclusive events and test drives for potential customers to experience the performance and eco-friendliness of EcoLux firsthand. These events can be held at upscale locations and feature interactive exhibits, product demonstrations, and networking opportunities.  These strategies will help position EcoLux as a leader in the luxury electric-vehicle market while appealing to environmentally conscious consumers who value innovation and sustainability. TERMINATE", "role": "user", "name": "Digital_Marketer"}]"""

In [11]:
user_proxy = autogen.UserProxyAgent(
    name="Chairperson",
    system_message="The chairperson for the meeting.",
    # code_execution_config={},
    code_execution_config=False,
    human_input_mode="TERMINATE",
)

cmo = autogen.AssistantAgent(
    name="Chief_Marketing_Officer",
    # system_message is used in the select speaker message
    description="The head of the marketing department working with the product manager and digital marketer to execute a strong marketing campaign for your car company.",
    # description is used to prompt the LLM as this agent
    system_message="You, Jane titled Chief_Marketing_Officer, or CMO, are the head of the marketing department and your objective is to guide your team to producing and marketing unique ideas for your luxury car models. Don't include your name at the start of your response or speak for any other team member, let them come up with their own ideas and strategies, speak just for yourself as the head of marketing. When yourself, the Product_Manager, and the Digital_Marketer have spoken and the meeting is finished, say TERMINATE to conclude the meeting.",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
    llm_config=gpt4_config,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    # system_message is used in the select speaker message
    description="Product head for the luxury model cars product line in the car company. Always coming up with new product enhancements for the cars.",
    # description is used to prompt the LLM as this agent
    system_message="You, Alice titled Product_Manager, are always coming up with new product enhancements for the luxury car models you look after. Review the meeting so far and respond with the answer to your current task.  Don't include your name at the start of your response and don't speak for anyone else, leave the Chairperson to pick the next person to speak.",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
    llm_config=gpt4_config,
)

digital = autogen.AssistantAgent(
    name="Digital_Marketer",
    # system_message is used in the select speaker message
    description="A seasoned digital marketer who comes up with online marketing strategies that highlight the key features of the luxury car models.",
    # description is used to prompt the LLM as this agent
    system_message="You, Elizabeth titled Digital_Marketer, are a senior online marketing specialist who comes up with marketing strategies that highlight the key features of the luxury car models.  Review the meeting so far and respond with the answer to your current task.   Don't include your name at the start of your response and don't speak for anyone else, leave the Chairperson to pick the next person to speak.",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
    llm_config=gpt4_config,
)

# Customised message, this is always the first message in the context
my_speaker_select_msg = """You are a chairperson for a marketing meeting for this car manufacturer where multiple members of the team will speak.
The job roles of the team at the meeting, and their responsibilities, are:
{roles}"""

# Customised prompt, this is always the last message in the context
my_speaker_select_prompt = """Read the above conversation.
Then select ONLY THE NAME of the next job role from {agentlist} to speak. Do not explain why."""

groupchat = autogen.GroupChat(
    agents=[user_proxy, cmo, pm, digital],
    messages=[],
    max_round=10,
    select_speaker_message_template=my_speaker_select_msg,
    select_speaker_prompt_template=my_speaker_select_prompt,
    max_retries_for_selecting_speaker=2,  # New
    select_speaker_auto_verbose=False,  # New
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=gpt4_config,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
)

In [12]:
# Prepare the group chat for resuming WITHOUT removing the TERMINATE message
last_agent, last_message = manager.resume(messages=previous_state)

# Resume and it will terminate immediately
result = last_agent.initiate_chat(recipient=manager, message=last_message, clear_history=False)

Prepared group chat with 4 messages, the last speaker is Digital_Marketer
Digital_Marketer (to chat_manager):

Thank you, Chief_Marketing_Officer!  For 'EcoLux', I propose the following three marketing strategies:  1. 'Green Revolution' Campaign: Highlighting the eco-friendly features of EcoLux through a series of social media ads and influencer partnerships. We can partner with eco-conscious influencers to showcase how EcoLux is not only a luxury car but also an environmentally responsible choice. 2. 'Fuel for Thought' Content Series: Creating a content series that explores the intersection of technology, sustainability, and luxury. This could include blog posts, videos, and podcasts that delve into the innovative features of EcoLux and its impact on the environment. 3. 'EcoLux Experience' Event Marketing: Hosting exclusive events and test drives for potential customers to experience the performance and eco-friendliness of EcoLux firsthand. These events can be held at upscale location

In [13]:
# Prepare the group chat for resuming WITH removal of TERMINATE message
last_agent, last_message = manager.resume(messages=previous_state, remove_termination_string="TERMINATE")

# Resume the chat using the last agent and message
# result = last_agent.initiate_chat(recipient=manager, message=last_message, clear_history=False)
chat_result = last_agent.initiate_chat(recipient=manager, message=last_message, clear_history=False)

# Output the final chat history showing the original 4 messages and the resumed message
for i, message in enumerate(groupchat.messages):
    print(
        f"#{i + 1}, {message['name']}: {message['content'][:80]}".replace("\n", " "),
        f"{'...' if len(message['content']) > 80 else ''}".replace("\n", " "),
    )

Prepared group chat with 4 messages, the last speaker is Digital_Marketer
Digital_Marketer (to chat_manager):

Thank you, Chief_Marketing_Officer!  For 'EcoLux', I propose the following three marketing strategies:  1. 'Green Revolution' Campaign: Highlighting the eco-friendly features of EcoLux through a series of social media ads and influencer partnerships. We can partner with eco-conscious influencers to showcase how EcoLux is not only a luxury car but also an environmentally responsible choice. 2. 'Fuel for Thought' Content Series: Creating a content series that explores the intersection of technology, sustainability, and luxury. This could include blog posts, videos, and podcasts that delve into the innovative features of EcoLux and its impact on the environment. 3. 'EcoLux Experience' Event Marketing: Hosting exclusive events and test drives for potential customers to experience the performance and eco-friendliness of EcoLux firsthand. These events can be held at upscale location

In [14]:
print(chat_result.summary)

Thank you, Digital_Marketer, for those insightful strategies. I believe we have a solid plan to move forward with 'EcoLux.' The combination of innovative product features and targeted marketing strategies will position us well in the luxury car market. Let's ensure we execute these ideas effectively to maximize our impact. 

If there are no further comments or questions, I will conclude the meeting. .


In [15]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

6
[{'content': "Let's get this meeting started. We'll have a set order of speakers. First the Product_Manager will create 3 new product ideas. Then the Chief_Marketing_Officer will speak and talk about the idea they have chosen to move forward with. Then the Digital_Marketer will create 3 marketing strategies for that idea. We MUST stick to the speaking order which is Product_Manager first, Chief_Marketing_Officer second, Digital_Marketer third, and finally the Chief_Marketing_Officer will speak and end the meeting.", 'name': 'Chairperson', 'role': 'user'}, {'content': "Sounds like a plan! Let's get started.  As the Product_Manager, I'd like to present three new product ideas for our luxury car models:  1. 'EcoLux': A hybrid electric-gasoline vehicle that combines the best of both worlds, offering exceptional fuel efficiency and reduced carbon footprint. 2. 'AeroSport': A high-performance sports car with advanced aerodynamics, designed to deliver unparalleled speed and agility on the t

## Example of resuming a terminated GroupChat with a new message and agent

In [16]:
# Prepare the group chat for resuming using the previous messages. We don't need to remove the TERMINATE string as we aren't using the last message for resuming.
last_agent, last_message = manager.resume(messages=groupchat.messages)

Prepared group chat with 6 messages, the last speaker is Chief_Marketing_Officer


In [17]:
# Resume the chat using a different agent and message
# result = manager.initiate_chat(
chat_result = manager.initiate_chat(
    recipient=cmo,
    message="Team, let's now think of a name for the next vehicle that embodies that idea. Chief_Marketing_Officer and Product_manager can you both suggest one and then we can conclude.",
    clear_history=False,
)

# Output the final chat history showing the original 4 messages and the resumed message
for i, message in enumerate(groupchat.messages):
    print(
        f"#{i + 1}, {message['name']}: {message['content'][:80]}".replace("\n", " "),
        f"{'...' if len(message['content']) > 80 else ''}".replace("\n", " "),
    )

chat_manager (to Chief_Marketing_Officer):

Team, let's now think of a name for the next vehicle that embodies that idea. Chief_Marketing_Officer and Product_manager can you both suggest one and then we can conclude.

--------------------------------------------------------------------------------
Chief_Marketing_Officer (to chat_manager):

For the next vehicle that embodies the 'EcoLux' idea, I suggest the name "EcoElite." This name conveys a sense of luxury while emphasizing the eco-friendly aspect of the vehicle. It positions the car as a premium choice for environmentally conscious consumers who do not want to compromise on luxury or performance.

Now, I would like to hear the Product_Manager's suggestion.

--------------------------------------------------------------------------------

Next speaker: Product_Manager

Product_Manager (to chat_manager):

For the 'EcoLux' concept, I propose the name "EcoElite" as well. This name effectively communicates the luxury and sophistication 

In [18]:
print(chat_result.summary)

Thank you all for your valuable contributions today. We have made significant progress in defining our new luxury vehicle, "EcoElite," and the marketing strategies to support it. I appreciate everyone's input and collaboration. 

.


In [19]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

12
[{'content': "Let's get this meeting started. We'll have a set order of speakers. First the Product_Manager will create 3 new product ideas. Then the Chief_Marketing_Officer will speak and talk about the idea they have chosen to move forward with. Then the Digital_Marketer will create 3 marketing strategies for that idea. We MUST stick to the speaking order which is Product_Manager first, Chief_Marketing_Officer second, Digital_Marketer third, and finally the Chief_Marketing_Officer will speak and end the meeting.", 'name': 'Chairperson', 'role': 'assistant'}, {'content': "Sounds like a plan! Let's get started.  As the Product_Manager, I'd like to present three new product ideas for our luxury car models:  1. 'EcoLux': A hybrid electric-gasoline vehicle that combines the best of both worlds, offering exceptional fuel efficiency and reduced carbon footprint. 2. 'AeroSport': A high-performance sports car with advanced aerodynamics, designed to deliver unparalleled speed and agility on